In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import numpy as np

# to ignore the warnings
from warnings import filterwarnings

In [34]:
total_df = pd.read_csv('./data/totalPredict_ML3.csv')
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9510 entries, 0 to 9509
Data columns (total 60 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   home_team                9510 non-null   object 
 1   visitor_team             9510 non-null   object 
 2   visitor_points           9510 non-null   float64
 3   visitor_fgm              9510 non-null   float64
 4   visitor_fga              9510 non-null   float64
 5   visitor_fgp              9510 non-null   float64
 6   visitor_ftm              9510 non-null   float64
 7   visitor_fta              9510 non-null   float64
 8   visitor_ftp              9510 non-null   float64
 9   visitor_tpm              9510 non-null   float64
 10  visitor_tpa              9510 non-null   float64
 11  visitor_tpp              9510 non-null   float64
 12  visitor_offReb           9510 non-null   float64
 13  visitor_defReb           9510 non-null   float64
 14  visitor_totReb          

In [35]:
total_df.isnull().sum()

home_team                  0
visitor_team               0
visitor_points             0
visitor_fgm                0
visitor_fga                0
visitor_fgp                0
visitor_ftm                0
visitor_fta                0
visitor_ftp                0
visitor_tpm                0
visitor_tpa                0
visitor_tpp                0
visitor_offReb             0
visitor_defReb             0
visitor_totReb             0
visitor_assists            0
visitor_pFouls             0
visitor_steals             0
visitor_turnovers          0
visitor_blocks             0
home_points                0
home_fgm                   0
home_fga                   0
home_fgp                   0
home_ftm                   0
home_fta                   0
home_ftp                   0
home_tpm                   0
home_tpa                   0
home_tpp                   0
home_offReb                0
home_defReb                0
home_totReb                0
home_assists               0
home_pFouls   

In [36]:
total_df.shape

(9510, 60)

In [37]:
winner_df = pd.read_csv('./data/winPredict_ML3.csv')


In [38]:
winner_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9510 entries, 0 to 9509
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   home_team                9510 non-null   object 
 1   visitor_team             9510 non-null   object 
 2   visitor_points           9510 non-null   float64
 3   visitor_fgm              9510 non-null   float64
 4   visitor_fga              9510 non-null   float64
 5   visitor_fgp              9510 non-null   float64
 6   visitor_ftm              9510 non-null   float64
 7   visitor_fta              9510 non-null   float64
 8   visitor_ftp              9510 non-null   float64
 9   visitor_tpm              9510 non-null   float64
 10  visitor_tpa              9510 non-null   float64
 11  visitor_tpp              9510 non-null   float64
 12  visitor_offReb           9510 non-null   float64
 13  visitor_defReb           9510 non-null   float64
 14  visitor_totReb          

In [39]:
winner_df.shape

(9510, 59)

In [61]:
# winner_df['is_home'] = (winner_df['home_team'] == winner_df['home_team']).astype(int)
# winner_df.head()

,home_team,visitor_team,visitor_points,visitor_fgm,visitor_fga,visitor_fgp,visitor_ftm,visitor_fta,visitor_ftp,visitor_tpm,...,home_DRtg,visitor_turn_over_ratio,home_turn_over_ratio,visitor_efg%,home_efg%,visitor_ts%,home_ts%,home_win,total_points,is_home
0,Houston Rockets,Los Angeles Lakers,142.0,53.0,105.0,0.505,26.0,37.0,0.703,10.0,...,119.167506,8.419533,6.741848,0.552381,0.547170,0.585422,0.600844,1,290.0,1
1,Sacramento Kings,Memphis Grizzlies,114.0,41.0,76.0,0.539,18.0,24.0,0.750,14.0,...,112.871287,12.845215,12.030075,0.631579,0.482558,0.658503,0.494845,0,210.0,1
2,LA Clippers,Charlotte Hornets,98.0,36.0,93.0,0.387,15.0,19.0,0.789,11.0,...,100.122599,12.026033,12.511730,0.446237,0.539474,0.483425,0.603095,1,204.0,1
3,Oklahoma City Thunder,Dallas Mavericks,116.0,46.0,89.0,0.517,9.0,14.0,0.643,15.0,...,117.029863,5.461923,8.585701,0.601124,0.539326,0.609500,0.570016,0,229.0,1
4,Phoenix Suns,Philadelphia 76ers,123.0,43.0,77.0,0.558,27.0,34.0,0.794,10.0,...,114.996260,15.332944,11.682243,0.623377,0.477273,0.668769,0.534188,0,233.0,1


# One-Hot Encoding

In [40]:
# Apply one-hot encoding to categorical columns
encoded_total_df = pd.get_dummies(total_df, columns=['visitor_team', 'home_team'])
encoded_winner_df = pd.get_dummies(winner_df, columns=['visitor_team', 'home_team'])

# Covert booleans to integers
new_total_df = encoded_total_df.replace({True: 1, False: 0})
new_winner_df = encoded_winner_df.replace({True: 1, False: 0})

C:\Users\User\AppData\Local\Temp\ipykernel_1316\1442891978.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_total_df = encoded_total_df.replace({True: 1, False: 0})
C:\Users\User\AppData\Local\Temp\ipykernel_1316\1442891978.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_winner_df = encoded_winner_df.replace({True: 1, False: 0})


In [41]:
new_winner_df = new_winner_df.dropna()
new_total_df = new_total_df.dropna()

In [42]:
print(f"Shape of encoded_total_df: {new_total_df.shape}")
print(f"Shape of encoded_winner_df: {new_winner_df.shape}")

Shape of encoded_total_df: (9509, 118)
Shape of encoded_winner_df: (9509, 117)


# Machine Learning

In [43]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Create feature values
X_total = new_total_df.drop(columns=['home_points', 'visitor_points', 'total_points', 'home_win'])
X_winner = new_winner_df.drop(columns=['home_points', 'visitor_points', 'total_points', 'home_win'])

# Create target values
y_total = new_total_df['total_points']
y_winner = new_winner_df['home_win']

# Split training data
X_total_train, X_total_test, y_total_train, y_total_test = train_test_split(
    X_total, 
    y_total, 
    test_size=0.2, 
    random_state=42
)

X_winner_train, X_winner_test, y_winner_train, y_winner_test = train_test_split(
    X_winner, 
    y_winner, 
    test_size=0.2, 
    random_state=42
)

# Initiate Scaler
scaler = StandardScaler()

# Train the classification model (winner predictions)
clf_model = DecisionTreeClassifier().fit(X_winner_train, y_winner_train)

# Scale training data
X_total_train_scaled = scaler.fit_transform(X_total_train)

# Train regression model (total prediction)
linear_model = LinearRegression().fit(X_total_train_scaled, y_total_train)

# Scale the test data
X_total_test_scaled = scaler.transform(X_total_test)

In [44]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report # type: ignore

# Evaluate the classification model
y_winner_pred = clf_model.predict(X_winner_test)
print(confusion_matrix(y_winner_test, y_winner_pred))
print(classification_report(y_winner_test, y_winner_pred))
print("Accuracy:", accuracy_score(y_winner_test, y_winner_pred))


[[917   9]
 [ 11 965]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       926
           1       0.99      0.99      0.99       976

    accuracy                           0.99      1902
   macro avg       0.99      0.99      0.99      1902
weighted avg       0.99      0.99      0.99      1902

Accuracy: 0.9894847528916929


In [45]:
y_total_pred = linear_model.predict(X_total_test_scaled)
from sklearn.metrics import mean_squared_error, r2_score
print("MSE:", mean_squared_error(y_total_test, y_total_pred))
print("R2 Score:", r2_score(y_total_test, y_total_pred))


MSE: 0.09779935099966582
R2 Score: 0.9997874950083913


In [46]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf_model, X_winner, y_winner, cv=5)
print("Cross-Validation Scores:", scores)
print("Mean CV Accuracy:", scores.mean())


Cross-Validation Scores: [0.98685594 0.98002103 0.98790747 0.99001052 0.98632299]
Mean CV Accuracy: 0.9862235881164985


In [ ]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }
# grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
# grid_search.fit(X_winner_train, y_winner_train)
# print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10}


In [62]:
# Inputs for the home and visitor teams
visitor_team = input("Enter Away Team: ")
home_team = input("Enter Home Team: ")

# Create input data with correct features for the models
input_data_winner = {col: 0 for col in X_winner.columns}
input_data_total = {col: 0 for col in X_total.columns}

# Populate the features based on the provided teams
if f'home_team_{home_team}' in input_data_winner:
    input_data_winner[f'home_team_{home_team}'] = 1
if f'visitor_team_{visitor_team}' in input_data_winner:
    input_data_winner[f'visitor_team_{visitor_team}'] = 1
    
    
if f'home_team_{home_team}' in input_data_total:
    input_data_winner[f'home_team_{home_team}'] = 1    
if f'visitor_team_{visitor_team}' in input_data_total:
    input_data_winner[f'visitor_team_{visitor_team}'] = 1
    
# Convert dictionaries to DataFrames
input_df_winner = pd.DataFrame([input_data_winner])
input_df_total = pd.DataFrame([input_data_total])

# Scale the input data for the total points
input_df_total_scaled = scaler.transform(input_df_total)

# Predict the winner using the classification model
predicted_winner = clf_model.predict(input_df_winner)[0]
predict_total_points = linear_model.predict(input_df_total_scaled)[0]

# Map the winner prediction to the team name
winner = home_team if predicted_winner == 1 else visitor_team

print("\n--- Game Prediction ---")
print(f"Predicted Winner: {winner}")
# print(f"Predicted Total Points: {round(predict_total_points, 0)}")



--- Game Prediction ---
Predicted Winner: Toronto Raptors
